In [1]:
# Download NOAA WL data using API calls and do a quick inventory
import pandas as pd
import os
import glob
import requests

# Choose where to put the data
#data_dir = 'F:/crs/proj/2025_NOPP_comparison/helene_water_level/noaa_station_data_hourly/'
data_dir = 'F:/crs/proj/2025_NOPP_comparison/helene_water_level/noaa_station_data_6min/'

# Choose whether hourly or 6-min records
# product = "hourly_height"
product = "water_level" #6-min records

def download_noaa_data(stations, begin_date, end_date, product = "hourly_height", save_dir = 'noaa_data'):
    """
    Downloads NOAA water level data for given station numbers and saves them as CSV files.
    
    Parameters:
    stations (list): List of NOAA station numbers.
    begin_date (str): Start date in YYYYMMDD format.
    end_date (str): End date in YYYYMMDD format.
    save_dir (str): Directory to save downloaded CSV files.
    """
    base_url = "https://api.tidesandcurrents.noaa.gov/api/prod/datagetter"
    params = {
        "product": product,
        "application": "NOS.COOPS.TAC.WL",
        "datum": "NAVD",
        "time_zone": "GMT",
        "units": "metric",
        "format": "csv"
    }
    
    # Ensure save directory exists
    os.makedirs(save_dir, exist_ok=True)
    
    for station in stations:
        params.update({"station": station, "begin_date": begin_date, "end_date": end_date})
        response = requests.get(base_url, params=params)
        
        if response.status_code == 200:
            file_path = os.path.join(save_dir, f"{station}_{begin_date}_{end_date}.csv")
            with open(file_path, "wb") as file:
                file.write(response.content)
            print(f"Downloaded: {file_path}")
        else:
            print(f"Failed to download data for station {station}: {response.status_code}")

def process_noaa_data(save_dir, output_file="summary.csv"):
    """
    Processes NOAA CSV files and extracts the number of records, 
    time of max water level, and value of max water level for each station.
    Saves the summary to a CSV file.
    """
    summary = []
    
    for file in os.listdir(save_dir):
        if file.endswith(".csv"):
            file_path = os.path.join(save_dir, file)
            station = file.split("_")[0]  # Extract station number from filename
            
            df = pd.read_csv(file_path)
            df.columns = df.columns.str.strip()  # Remove any leading/trailing spaces in column names
            
            if "Water Level" in df.columns and "Date Time" in df.columns:
                df = df.dropna(subset=["Water Level"])  # Remove rows with missing water level data
                
                if df.empty:
                    num_records = 0
                    max_time = float('nan')
                    max_value = float('nan')
                else:
                    num_records = len(df)
                    max_row = df.loc[df["Water Level"].idxmax()]
                    max_time = max_row["Date Time"]
                    max_value = max_row["Water Level"]
                
                summary.append({
                    "Station": station,
                    "Num Records": num_records,
                    "Max Time": max_time,
                    "Max Value": max_value
                })
    
    summary_df = pd.DataFrame(summary)
    summary_df.to_csv(output_file, index=False)
    print(f"Summary saved to {output_file}")
    return summary

# These stations were determined from the Flood Event Viewer map
stations = [
    "8723970", "8724580", "8725110", "8725114", "8725520",
    "8726384", "8726520", "8726607", "8726674", "8726724",
    "8727520", "8728690", "8729108", "8729210", "8729840"
]
begin_date = "20220927"
end_date = "20220930"
download_noaa_data(stations, begin_date, end_date, product = "water_level", save_dir = data_dir)
summary = process_noaa_data(save_dir = data_dir, output_file = data_dir+'summary_6min.csv')
for entry in summary:
    print(entry)


Downloaded: F:/crs/proj/2025_NOPP_comparison/helene_water_level/noaa_station_data_6min/8723970_20220927_20220930.csv
Downloaded: F:/crs/proj/2025_NOPP_comparison/helene_water_level/noaa_station_data_6min/8724580_20220927_20220930.csv
Downloaded: F:/crs/proj/2025_NOPP_comparison/helene_water_level/noaa_station_data_6min/8725110_20220927_20220930.csv
Downloaded: F:/crs/proj/2025_NOPP_comparison/helene_water_level/noaa_station_data_6min/8725114_20220927_20220930.csv
Downloaded: F:/crs/proj/2025_NOPP_comparison/helene_water_level/noaa_station_data_6min/8725520_20220927_20220930.csv
Downloaded: F:/crs/proj/2025_NOPP_comparison/helene_water_level/noaa_station_data_6min/8726384_20220927_20220930.csv
Downloaded: F:/crs/proj/2025_NOPP_comparison/helene_water_level/noaa_station_data_6min/8726520_20220927_20220930.csv
Downloaded: F:/crs/proj/2025_NOPP_comparison/helene_water_level/noaa_station_data_6min/8726607_20220927_20220930.csv
Downloaded: F:/crs/proj/2025_NOPP_comparison/helene_water_level/